In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

# http://pytorch.org/
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision
import torch
!pip install torchsummary

!pip install Pillow==4.0.0
!pip install PIL
!pip install image
from PIL import Image
import pickle

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import math;

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable

from torch.utils.data import Dataset, DataLoader,TensorDataset
from torchvision import transforms

from torchsummary import summary
%matplotlib inline

import os
os.chdir("/content/drive/My Drive/code for convolocean")
os.getcwd()
os.listdir(os.getcwd())

In [ ]:
import pandas as pd
import numpy as np

from random import shuffle
from torch.optim.lr_scheduler import ReduceLROnPlateau


In [ ]:

train_images = pickle.load(open(os.getcwd()+"/pr_train_images.pkl","rb"))
train_labels = pickle.load(open(os.getcwd()+"/pr_labels.pkl","rb"))

test_images  = pickle.load(open(os.getcwd()+"/pr_test_img.pkl","rb"))
test_labels  = pickle.load(open(os.getcwd()+"/pr_test_labels.pkl","rb"))

val_images   = pickle.load(open(os.getcwd()+"/pr_val_img.pkl","rb"))
val_fname    = pickle.load(open(os.getcwd()+"/val_nam.pkl","rb"))

In [ ]:
n_test_images=[]
n_test_labels=[]
i=0
for image in test_images:
    n_test_images.append(image)
    n_test_images.append(transforms.functional.rotate(image, 90))
    n_test_images.append(transforms.functional.rotate(image, 180))
    n_test_images.append(transforms.functional.rotate(image, 270))
    n_test_labels.extend(4*[test_labels[i]])
    i+=1

In [ ]:
n_val_images=[]
n_val_fname=[]
i=0
for image in val_images:
    n_val_images.append(image)
    n_val_images.append(transforms.functional.rotate(image, 90))
    n_val_images.append(transforms.functional.rotate(image, 180))
    n_val_images.append(transforms.functional.rotate(image, 270))
    #n_val_fname.extend(4*[val_fname[i]])
    i+=1

In [ ]:
class TrainingImagesDataset(Dataset):
    
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform
        
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, index):
        single_image = self.images[index]
        single_label = self.labels[index]
        
        if self.transform != None:
            #image to tensor
            tensor = self.transform(single_image)
            return (tensor, single_label)
        #return image without turning it to tensor
        #probably wrong
        return (single_image, single_label)

In [ ]:

class ValidationImagesDataset(Dataset):
    
    def __init__(self, images, transform=None):
        self.images = images
        self.transform = transform
    
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, index):
        single_image = self.images[index]
        if self.transform:
            #image to tensor
            tensor = self.transform(single_image)
            return tensor
        return single_image

In [ ]:
def create_dataloaders(train_images, train_lbs, test_images=None, test_lbs = None, batch_size = 32):
    
    train_dataset = TrainingImagesDataset(train_images, train_lbs, transform=transforms.Compose([transforms.RandomHorizontalFlip(p=0.5),transforms.RandomVerticalFlip(p=0.5),transforms.RandomRotation(degrees=360), transforms.ToTensor()]))
    
    if test_images and test_lbs:
        print("here")
        test_dataset = TrainingImagesDataset(test_images, test_lbs, transform=transforms.Compose([transforms.ToTensor()]))
    elif test_images:
        test_dataset = ValidationImagesDataset(test_images, transform=transforms.Compose([transforms.ToTensor()]))
    else:
        test_dataset = []
    
    train_loader = DataLoader(train_dataset, batch_size = batch_size, shuffle = True, num_workers = 0)
    test_loader = DataLoader(test_dataset, batch_size = batch_size, shuffle = False, num_workers = 0)    
    
    return (train_loader, test_loader)

In [ ]:
lass CNN(nn.Module):
    
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 12, 5, padding=2)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(12, 32, 3, padding=1)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(32, 48, 3, padding=1)
        #self.pool3 = nn.MaxPool2d(2,2)
        self.conv4 = nn.Conv2d(48,32,3, padding=1)
        self.pool3 = nn.MaxPool2d(2,2)
        self.fc1 = nn.Linear(2048, 2048)
        self.fc2 = nn.Linear(2048, 2048)
        self.fc3 = nn.Linear(2048, 121)

    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        x = F.relu(self.conv3(x))
        x = self.pool3(F.relu(self.conv4(x)))
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features
ocean = CNN().cuda()

In [ ]:

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(ocean.parameters(), lr = 0.001, momentum = 0.9, weight_decay=0.0005)
lr_scheduler = ReduceLROnPlateau(optimizer, 'max', factor = 0.1, patience = 5, verbose=True)

train_loader, test_loader = create_dataloaders(train_images, train_labels,n_test_images,n_test_labels)

In [ ]:
%%time
EPOCH = 10
for epoch in range(EPOCH):  # loop over the dataset multiple times
    ocean.train().cuda()
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(train_loader, 0):
        # get the inputs
        inputs = inputs.cuda()
        lbs = labels.cuda()

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = ocean(inputs)
        loss = criterion(outputs, lbs)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 1000 == 999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 1000))
            running_loss = 0.0
    ocean.eval().cuda()
    correct = 0
    total = 0
    for (images, labels) in test_loader:
        images = images.cuda()
        labels=labels[::4]
        labels = labels.cuda()
        #print(images)
        outputs = ocean(images)
        outputs1=[]
        for i in range(0,len(outputs),4):
            c = torch.sum(outputs[i:i+4],0)
            outputs1.append(c)
        outputs1=torch.stack(outputs1)
        _, predicted = torch.max(outputs1.data, 1)
        
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    print('Accuracy on EPOCH %d test images: %d %%' % (epoch+1, 100 * correct / total))   
    lr_scheduler.step(correct / total)
print('Finished Training')

In [ ]:
correct = 0
total = 0
jj=0
with torch.no_grad():
    try:
        for (images, labels) in test_loader:
            try:
                jj+=1
                #print("hi")
                images = images.cuda()
                labels = labels.cuda()
                #print(images)
                #input()
                outputs = ocean(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
                #print("here")
            except:
                print(jj)
    except:
        print(jj)
print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))
print(correct)
print(total)

In [ ]:
val_dataset = ValidationImagesDataset(n_val_images, transform=transforms.Compose([transforms.ToTensor()]))
val_loader = DataLoader(val_dataset, batch_size = 32, shuffle = False, num_workers = 0)

In [ ]:
prediction = []
i=0
for images in val_loader:
    i+=1
    outputs = ocean(images.cuda())
    #print(outputs.size())
    #print(outputs[0])
    #input()outputs1=[]
    for i in range(0,len(outputs),4):
        c = torch.sum(outputs[i:i+4],0)
        outputs1.append(c)
    outputs1=torch.stack(outputs1)
    _, predicted = torch.max(outputs1.data, 1)
    prediction.extend(predicted.cpu().numpy())
print(i)

In [ ]:
results_df = pd.DataFrame({'image': val_fname, 'class': prediction}, columns=['image', 'class'])
results_df.to_csv(os.getcwd()+"/results_new.csv", sep = ',', index = False)